# 6. Modelos de Regressão


Este notebook explora diferentes modelos de regressão para análise de dados, incluindo Regressão Linear e Lasso, utilizando técnicas de pré-processamento como transformação Box-Cox e seleção de variáveis. O objetivo é avaliar o desempenho dos modelos em termos de métricas como RMSE e R², tanto nos datasets completos quanto após a remoção de outliers das questões do Enem (vetorizadas).


In [1]:
# Importando Dependências para Modelos de Regressão
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LassoCV

In [2]:
# Leitura dos dados
enem_data = pd.read_csv("../data/final/enem_data_embeddings_v2.csv")
enem_data.head()

,Unnamed: 0,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,...,[-1.73957720e-02 -4.40281304e-03 -4.69083376e-...,[-1.63012251e-01 -1.42375007e-01 1.21714994e-...,[-0.041816 -0.06492707 0.0900044 -0.049802...,0.141353,0.516898,0.151251,C,True,C,True
1,1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,...,[-5.19490764e-02 3.12845111e-02 -7.22410753e-...,[-0.18672037 0.04755275 -0.17406088 -0.039877...,[ 0.05533243 -0.03906671 -0.05943896 0.019810...,0.422576,0.313321,0.423611,D,True,D,True
2,2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,...,[ 0.02604909 0.04325113 -0.02975703 -0.001379...,[ 1.17409984e-02 7.59263411e-02 -1.61446687e-...,[ 6.66799322e-02 1.58033855e-02 2.94438470e-...,0.427063,0.471643,0.602545,D,True,D,True
3,3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,...,[-5.25127212e-03 3.98711637e-02 -1.80381946e-...,[-0.07366925 0.12609875 0.063628 0.126195...,[-0.07921684 0.10788742 0.10466367 -0.041218...,0.320939,0.482461,0.421501,E,True,E,True
4,4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,...,[ 1.73867792e-02 3.37337283e-03 -2.72473246e-...,[ 0.11709199 -0.16062833 0.08286966 -0.119532...,[-2.20553298e-02 8.12978372e-02 1.06412418e-...,0.304050,0.501346,0.411802,C,True,C,True


In [3]:
enem_data = enem_data[enem_data["nu_param_B"].notna()]

---

## 6.1. Embeddings de 300 Dimensões


In [4]:
# Coletando os dados
X_300 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_300"]
]  # Somente Embeddings 300

y_300 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [5]:
# Aplicando Transformações
add_list = [(y_300.min() * (-1)) + 1] * len(y_300)
y_300 = y_300 + add_list

# Aplicando Box-Cox
y_300, best_lambda = stats.boxcox(y_300)
print(best_lambda)

0.6304389770836538


### 6.1.1. Regressão Linear


In [6]:
# Separando em conjunto de treino e teste
X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(
    X_300, y_300, test_size=0.3, random_state=42
)

In [7]:
# Adicionando constante
X_train_300 = sm.add_constant(X_train_300)
X_test_300 = sm.add_constant(X_test_300)

UFuncTypeError: ufunc 'maximum' did not contain a loop with signature matching types (dtype('<U4875'), dtype('<U4875')) -> None

In [ ]:
# Criando o modelo e realizando predição
linear_model_300 = sm.OLS(y_train_300, X_train_300).fit()
prediction_linear_model_300 = linear_model_300.predict(X_test_300)

In [ ]:
# Cálculo do RMSE
rmse_300 = root_mean_squared_error(y_test_300, prediction_linear_model_300)
print("RMSE: ", rmse_300)

In [ ]:
# Visualização do modelo e resultados
print(linear_model_300.summary(alpha=0.05))

### 6.1.2. Regressão Lasso


In [ ]:
# Separando em conjunto de treino e teste
X_train_300_lasso, X_test_300_lasso, y_train_300_lasso, y_test_300_lasso = (
    train_test_split(X_300, y_300, test_size=0.3, random_state=42)
)

In [ ]:
lasso_model_300 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_300_lasso, y_train_300_lasso
)
pred_lasso_model_300 = lasso_model_300.predict(X_test_300_lasso)

print("Melhor R2 Score:", lasso_model_300.score(X_train_300_lasso, y_train_300_lasso))
print("Melhor Alpha:", lasso_model_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso, pred_lasso_model_300),
)

### 6.1.3. Regressões com Corte (-3,3)


In [ ]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

In [ ]:
# Coletando os dados
X_300_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_300"]
]
y_300_filtered = enem_filtered["nu_param_B"]

In [ ]:
# Aplicando Transformações
add_list = [(y_300_filtered.min() * (-1)) + 1] * len(y_300_filtered)
y_300_filtered = y_300_filtered + add_list

# Aplicando Boxcox
y_300_filtered, best_lambda = stats.boxcox(y_300_filtered)
print(best_lambda)

### 6.1.3.A) Regressão Linear


In [ ]:
# Dividindo os dados em treino e teste
X_train_300_filtered, X_test_300_filtered, y_train_300_filtered, y_test_300_filtered = (
    train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)
)

In [ ]:
# Adicionando constante
X_train_300_filtered = sm.add_constant(X_train_300_filtered)
X_test_300_filtered = sm.add_constant(X_test_300_filtered)

In [ ]:
# Criando o modelo e realizando a predição
linear_model_300_filtered = sm.OLS(y_train_300_filtered, X_train_300_filtered).fit()
pred_linear_model_300_filtered = linear_model_300_filtered.predict(X_test_300_filtered)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_filtered, pred_linear_model_300_filtered)
print("RMSE:", rms)

In [ ]:
print(linear_model_300_filtered.summary(alpha=0.05))

### 6.1.3.B) Regressão Lasso


In [ ]:
# Dividindo os dados em treino e teste
(
    X_train_300_filtered_lasso,
    X_test_300_filtered_lasso,
    y_train_300_filtered_lasso,
    y_test_300_filtered_lasso,
) = train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)

In [ ]:
lasso_model_300_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_300_filtered_lasso, y_train_300_filtered_lasso)

pred_300_lasso_filtered = lasso_model_300_filtered.predict(X_test_300_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_300_filtered.score(
        X_train_300_filtered_lasso, y_train_300_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_300_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_300_filtered_lasso, pred_300_lasso_filtered),
)

### 6.1.4. LLM & Similaridade Features


In [ ]:
# Coletando os dados
y_300_all_features = enem_data["nu_param_B"]
X_300_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_300,
                similaridade_enunciado_distratores_300,
                similaridade_gabarito_distratores_300,
            ],
        ]
    )
    for embedding, similaridade_enunciado_gabarito_300, similaridade_enunciado_distratores_300, similaridade_gabarito_distratores_300 in zip(
        enem_data["enunciado_embbedings_word2vec_300"],
        enem_data["similaridade_enunciado_gabarito_300"],
        enem_data["similaridade_enunciado_distratores_300"],
        enem_data["similaridade_gabarito_distratores_300"],
    )
]

In [ ]:
# Aplicando Transformações
add_list = [(y_300_all_features.min() * (-1)) + 1] * len(y_300_all_features)
y_300_all_features = y_300_all_features + add_list

# Aplicando Boxcox
y_300_all_features, best_lambda = stats.boxcox(y_300_all_features)
print(best_lambda)

### 6.1.4.A) Regressão linear


In [ ]:
# Separando em conjunto de treino e teste

(
    X_train_300_all_features,
    X_test_300_all_features,
    y_train_300_all_features,
    y_test_300_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [ ]:
# Adicionando constante
X_train_300_all_features = sm.add_constant(X_train_300_all_features)
X_test_300_all_features = sm.add_constant(X_test_300_all_features)

In [ ]:
# Criando o modelo e realizando predição
linear_model_300_all_features = sm.OLS(
    y_train_300_all_features, X_train_300_all_features
).fit()
pred_linear_model_300_all_features = linear_model_300_all_features.predict(
    X_test_300_all_features
)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_all_features, pred_linear_model_300_all_features)
print("RMSE", rms)

In [ ]:
print(linear_model_300_all_features.summary(alpha=0.05))

### 6.1.4.B) Lasso


In [ ]:
# Separando em conjunto de treino e teste
(
    X_train_300_lasso_all_features,
    X_test_300_lasso_all_features,
    y_train_300_lasso_all_features,
    y_test_300_lasso_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [ ]:
lasso_model_all_features_300 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_300_lasso_all_features, y_train_300_lasso_all_features)

pred_300_lasso_all_features = lasso_model_all_features_300.predict(
    X_test_300_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_300.score(
        X_train_300_lasso_all_features, y_train_300_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso_all_features, pred_300_lasso_all_features),
)

---

## 6.2. Embeddings de 100 Dimensões


In [ ]:
# Coletando os dados
X_100 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_100"]
]  # Somente Embeddings

y_100 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [ ]:
# Aplicando Transformações
add_list = [(y_100.min() * (-1)) + 1] * len(y_100)
y_100 = y_100 + add_list

# Aplicando Box-Cox
y_100, best_lambda = stats.boxcox(y_100)
print(best_lambda)

### 6.2.1. Regressão Linear


In [ ]:
# Separando em conjunto de treino e teste
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(
    X_100, y_100, test_size=0.3, random_state=42
)

In [ ]:
# Adicionando constante
X_train_100 = sm.add_constant(X_train_100)
X_test_100 = sm.add_constant(X_test_100)

In [ ]:
# Criando o modelo e realizando predição
linear_model_100 = sm.OLS(y_train_100, X_train_100).fit()
pred_linear_model_100 = linear_model_100.predict(X_test_100)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100, pred_linear_model_100)
print("RMSE: ", rms)

In [ ]:
# Visualização do modelo e resultados
print(linear_model_100.summary(alpha=0.05))

### 6.2.2. Regressão Lasso


In [ ]:
# Separando em conjunto de treino e teste
X_train_100_lasso, X_test_100_lasso, y_train_100_lasso, y_test_100_lasso = (
    train_test_split(X_100, y_100, test_size=0.3, random_state=42)
)

In [ ]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_100_lasso, y_train_100_lasso
)
pred_lasso = model_lasso.predict(X_test_100_lasso)

print("Melhor R2 Score:", model_lasso.score(X_train_100_lasso, y_train_100_lasso))
print("Melhor Alpha:", model_lasso.alpha_)
print(
    "RMSE com o alpha escolhido:", root_mean_squared_error(y_test_100_lasso, pred_lasso)
)

### 6.2.3. Regressões com Corte (-3,3)


In [ ]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

In [ ]:
# Coletando os dados
X_100_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_100"]
]
y_100_filtered = enem_filtered["nu_param_B"]

In [ ]:
# Aplicando Transformações
add_list = [(y_100_filtered.min() * (-1)) + 1] * len(y_100_filtered)
y_100_filtered = y_100_filtered + add_list

# Aplicando Boxcox
y_100_filtered, best_lambda = stats.boxcox(y_100_filtered)
print(best_lambda)

### 6.2.3.A) Regressão Linear


In [ ]:
# Dividindo os dados em treino e teste
X_train_100_filtered, X_test_100_filtered, y_train_100_filtered, y_test_100_filtered = (
    train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)
)

In [ ]:
# Adicionando constante
X_train_100_filtered = sm.add_constant(X_train_100_filtered)
X_test_100_filtered = sm.add_constant(X_test_100_filtered)

In [ ]:
# Criando o modelo e realizando a predição
linear_model_100_filtered = sm.OLS(y_train_100_filtered, X_train_100_filtered).fit()
pred_linear_model_100_filtered = linear_model_100_filtered.predict(X_test_100_filtered)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_filtered, pred_linear_model_100_filtered)
print("RMSE:", rms)

In [ ]:
print(linear_model_100_filtered.summary(alpha=0.05))

### 6.2.3.B) Regressão Lasso


In [ ]:
# Dividindo os dados em treino e teste
(
    X_train_100_filtered_lasso,
    X_test_100_filtered_lasso,
    y_train_100_filtered_lasso,
    y_test_100_filtered_lasso,
) = train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)

In [ ]:
lasso_model_100_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_100_filtered_lasso, y_train_100_filtered_lasso)

pred_lasso_filtered_100 = lasso_model_100_filtered.predict(X_test_100_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_100_filtered.score(
        X_train_100_filtered_lasso, y_train_100_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_100_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_100_filtered_lasso, pred_lasso_filtered_100),
)

### 6.2.4. LLM & Similaridade Features


In [ ]:
# Coletando os dados
y_100_all_features = enem_data["nu_param_B"]
X_100_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_100,
                similaridade_enunciado_distratores_100,
                similaridade_gabarito_distratores_100,
            ],
        ]
    )
    for embedding, similaridade_enunciado_gabarito_100, similaridade_enunciado_distratores_100, similaridade_gabarito_distratores_100 in zip(
        enem_data["enunciado_embbedings_word2vec_100"],
        enem_data["similaridade_enunciado_gabarito_100"],
        enem_data["similaridade_enunciado_distratores_100"],
        enem_data["similaridade_gabarito_distratores_100"],
    )
]

In [ ]:
# Aplicando Transformações
add_list = [(y_100_all_features.min() * (-1)) + 1] * len(y_100_all_features)
y_100_all_features = y_100_all_features + add_list

# Aplicando Boxcox
y_100_all_features, best_lambda = stats.boxcox(y_100_all_features)
print(best_lambda)

### 6.2.4.A) Regressão linear


In [ ]:
# Separando em conjunto de treino e teste

(
    X_train_100_all_features,
    X_test_100_all_features,
    y_train_100_all_features,
    y_test_100_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [ ]:
# Adicionando constante
X_train_100_all_features = sm.add_constant(X_train_100_all_features)
X_test_100_all_features = sm.add_constant(X_test_100_all_features)

In [ ]:
# Criando o modelo e realizando predição
linear_model_100_all_features = sm.OLS(
    y_train_100_all_features, X_train_100_all_features
).fit()
pred_linear_model_100_all_features = linear_model_100_all_features.predict(
    X_test_100_all_features
)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_all_features, pred_linear_model_100_all_features)
print("RMSE", rms)

In [ ]:
print(linear_model_100_all_features.summary(alpha=0.05))

### 6.2.4.B) Lasso


In [ ]:
# Separando em conjunto de treino e teste
(
    X_train_100_lasso_all_features,
    X_test_100_lasso_all_features,
    y_train_100_lasso_all_features,
    y_test_100_lasso_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [ ]:
lasso_model_all_features_100 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_100_lasso_all_features, y_train_100_lasso_all_features)

pred_100_lasso_all_features = lasso_model_all_features_100.predict(
    X_test_100_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_100.score(
        X_train_100_lasso_all_features, y_train_100_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_100.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_100_lasso_all_features, pred_100_lasso_all_features),
)

---

## 6.3. Embeddings de 50 Dimensões


In [ ]:
# Coletando os dados
X_50 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_50"]
]  # Somente Embeddings 50

y_50 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [ ]:
# Aplicando Transformações
add_list = [(y_50.min() * (-1)) + 1] * len(y_50)
y_50 = y_50 + add_list

# Aplicando Box-Cox
y_50, best_lambda = stats.boxcox(y_50)
print(best_lambda)

### 6.3.1. Regressão Linear


In [ ]:
# Separando em conjunto de treino e teste
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [ ]:
# Adicionando constante
X_train_50 = sm.add_constant(X_train_50)
X_test_50 = sm.add_constant(X_test_50)

In [ ]:
# Criando o modelo e realizando predição
linear_model_50 = sm.OLS(y_train_50, X_train_50).fit()
pred_linear_model_50 = linear_model_50.predict(X_test_50)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50, pred_linear_model_50)
print("RMSE: ", rms)

In [ ]:
# Visualização do modelo e resultados
print(linear_model_50.summary(alpha=0.05))

### 6.3.2. Regressão Lasso


In [ ]:
# Separando em conjunto de treino e teste
X_train_50_lasso, X_test_50_lasso, y_train_50_lasso, y_test_50_lasso = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [ ]:
lasso_model_50 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_50_lasso, y_train_50_lasso
)
pred_lasso_model_50 = lasso_model_50.predict(X_test_50_lasso)

print("Melhor R2 Score:", lasso_model_50.score(X_train_50_lasso, y_train_50_lasso))
print("Melhor Alpha:", lasso_model_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso, pred_lasso_model_50),
)

### 6.3.3. Regressões com Corte (-3,3)


In [ ]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

In [ ]:
# Coletando os dados
X_50_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_50"]
]
y_50_filtered = enem_filtered["nu_param_B"]

In [ ]:
# Aplicando Transformações
add_list = [(y_50_filtered.min() * (-1)) + 1] * len(y_50_filtered)
y_50_filtered = y_50_filtered + add_list

# Aplicando Boxcox
y_50_filtered, best_lambda = stats.boxcox(y_50_filtered)
print(best_lambda)

### 6.3.3.A) Regressão Linear


In [ ]:
# Dividindo os dados em treino e teste
X_train_50_filtered, X_test_50_filtered, y_train_50_filtered, y_test_50_filtered = (
    train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)
)

In [ ]:
# Adicionando constante
X_train_50_filtered = sm.add_constant(X_train_50_filtered)
X_test_50_filtered = sm.add_constant(X_test_50_filtered)

In [ ]:
# Criando o modelo e realizando a predição
linear_model_50_filtered = sm.OLS(y_train_50_filtered, X_train_50_filtered).fit()
pred_linear_model_50_filtered = linear_model_50_filtered.predict(X_test_50_filtered)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_filtered, pred_linear_model_50_filtered)
print("RMSE:", rms)

In [ ]:
print(linear_model_50_filtered.summary(alpha=0.05))

### 6.3.3.B) Regressão Lasso


In [ ]:
# Dividindo os dados em treino e teste
(
    X_train_50_filtered_lasso,
    X_test_50_filtered_lasso,
    y_train_50_filtered_lasso,
    y_test_50_filtered_lasso,
) = train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)

In [ ]:
lasso_model_50_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_50_filtered_lasso, y_train_50_filtered_lasso)

pred_lasso_model_50_filtered = lasso_model_50_filtered.predict(X_test_50_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_50_filtered.score(X_train_50_filtered_lasso, y_train_50_filtered_lasso),
)
print("Melhor Alpha:", lasso_model_50_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_50_filtered_lasso, pred_lasso_model_50_filtered),
)

### 6.3.4. LLM & Similaridade Features


In [ ]:
# Coletando os dados
y_50_all_features = enem_data["nu_param_B"]
X_50_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_50,
                similaridade_enunciado_distratores_50,
                similaridade_gabarito_distratores_50,
            ],
        ]
    )
    for embedding, similaridade_enunciado_gabarito_50, similaridade_enunciado_distratores_50, similaridade_gabarito_distratores_50 in zip(
        enem_data["enunciado_embbedings_word2vec_50"],
        enem_data["similaridade_enunciado_gabarito_50"],
        enem_data["similaridade_enunciado_distratores_50"],
        enem_data["similaridade_gabarito_distratores_50"],
    )
]

In [ ]:
# Aplicando Transformações
add_list = [(y_50_all_features.min() * (-1)) + 1] * len(y_50_all_features)
y_50_all_features = y_50_all_features + add_list

# Aplicando Boxcox
y_50_all_features, best_lambda = stats.boxcox(y_50_all_features)
print(best_lambda)

### 6.3.4.A) Regressão linear


In [ ]:
# Separando em conjunto de treino e teste

(
    X_train_50_all_features,
    X_test_50_all_features,
    y_train_50_all_features,
    y_test_50_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [ ]:
# Adicionando constante
X_train_50_all_features = sm.add_constant(X_train_50_all_features)
X_test_50_all_features = sm.add_constant(X_test_50_all_features)

In [ ]:
# Criando o modelo e realizando predição
linear_model_50_all_features = sm.OLS(
    y_train_50_all_features, X_train_50_all_features
).fit()
pred_linear_model_50_all_features = linear_model_50_all_features.predict(
    X_test_50_all_features
)

In [ ]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_all_features, pred_linear_model_50_all_features)
print("RMSE", rms)

In [ ]:
print(linear_model_50_all_features.summary(alpha=0.05))

### 6.3.4.B) Lasso


In [ ]:
# Separando em conjunto de treino e teste
(
    X_train_50_lasso_all_features,
    X_test_50_lasso_all_features,
    y_train_50_lasso_all_features,
    y_test_50_lasso_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [ ]:
lasso_model_all_features_50 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_50_lasso_all_features, y_train_50_lasso_all_features)

pred_50_lasso_all_features = lasso_model_all_features_50.predict(
    X_test_50_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_50.score(
        X_train_50_lasso_all_features, y_train_50_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso_all_features, pred_50_lasso_all_features),
)

---
